# Análisis EDA de productores beneficiarios 2023

# 1. Importación de librerías y carga de datos

In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
# pd.set_option('display.max_columns', None)

In [2]:
lista_beneficiarios = pd.read_csv('../../data/LISTADO_BENEFICIARIOS_COMPLETO.csv', dtype={'cve_mun': str, 'cve_ent': str})

# 2. Análisis Inicial

## 2.1 Estructura de los datos

In [3]:
lista_beneficiarios.columns

Index(['ESTADO_prod', 'MUNICIPIO_prod', 'ACUSE ESTATAL', 'APELLIDO PATERNO',
       'APELLIDO MATERNO', 'NOMBRE (S)', 'PAQUETE', 'KEY_benef_Verificado',
       'Entidad_inegi', 'Municipio_inegi', 'CVE_ENT', 'CVE_MUN'],
      dtype='object')

## 2.2 Información general

In [4]:
lista_beneficiarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682804 entries, 0 to 1682803
Data columns (total 12 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   ESTADO_prod           1682804 non-null  object 
 1   MUNICIPIO_prod        1682804 non-null  object 
 2   ACUSE ESTATAL         1682804 non-null  object 
 3   APELLIDO PATERNO      1682803 non-null  object 
 4   APELLIDO MATERNO      1656391 non-null  object 
 5   NOMBRE (S)            1682804 non-null  object 
 6   PAQUETE               1682796 non-null  float64
 7   KEY_benef_Verificado  1682804 non-null  object 
 8   Entidad_inegi         1682804 non-null  object 
 9   Municipio_inegi       1682804 non-null  object 
 10  CVE_ENT               1682804 non-null  int64  
 11  CVE_MUN               1682804 non-null  int64  
dtypes: float64(1), int64(2), object(9)
memory usage: 154.1+ MB


## 2.3 Estadísticas Descriptivas

In [5]:
# Obtener estadísticas descriptivas para todas las variables
descriptive_stats = lista_beneficiarios.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                          count   unique                             top  \
ESTADO_prod             1682804       30                        GUERRERO   
MUNICIPIO_prod          1682804     2237                  LAS MARGARITAS   
ACUSE ESTATAL           1682804  1680105  23-PRONAFE-FERT-094971-S000-OC   
APELLIDO PATERNO        1682803    14995                       HERNANDEZ   
APELLIDO MATERNO        1656391    17604                       HERNANDEZ   
NOMBRE (S)              1682804   124628                            JUAN   
PAQUETE               1682796.0      NaN                             NaN   
KEY_benef_Verificado    1682804     2334          chiapas-las margaritas   
Entidad_inegi           1682804       30                        Guerrero   
Municipio_inegi         1682804     2215                  Las Margaritas   
CVE_ENT               1682804.0      NaN                             NaN   
CVE_MUN               1682804.0      NaN                             NaN   

           

# 3. Limpieza de Datos

## 3.1 Valores Nulos

In [6]:
valores_nulos = lista_beneficiarios.isna().sum()
print(valores_nulos)

ESTADO_prod                 0
MUNICIPIO_prod              0
ACUSE ESTATAL               0
APELLIDO PATERNO            1
APELLIDO MATERNO        26413
NOMBRE (S)                  0
PAQUETE                     8
KEY_benef_Verificado        0
Entidad_inegi               0
Municipio_inegi             0
CVE_ENT                     0
CVE_MUN                     0
dtype: int64


## 3.2 Decisiones sobre valores nulos

Se decide mantener las 8 filas con paquetes nulos.

## 3.3 Valores Duplicados

In [7]:
# Filas completas duplicadas
duplicated_rows_beneficiarios = lista_beneficiarios.drop(columns = ['ACUSE ESTATAL'])

filas_duplicadas = duplicated_rows_beneficiarios.duplicated().sum()
print(f'Hay {filas_duplicadas} filas duplicadas')

Hay 34092 filas duplicadas


Consultamos el número de filas duplicadas para columna.

In [8]:
duplicated_rows_beneficiarios.shape

(1682804, 11)

In [11]:
# Valores duplicados en la columna nombre, apellido, municipio y estado
duplicated_rows = duplicated_rows_beneficiarios.duplicated(subset=['APELLIDO PATERNO', 'APELLIDO MATERNO', 'NOMBRE (S)', 'MUNICIPIO_prod', 'ESTADO_prod'])
number_of_duplicated_rows = duplicated_rows.sum()
print(f"El número de filas duplicadas en las columnas 'APELLIDO PATERNO', 'APELLIDO MATERNO', 'NOMBRE (S)', 'MUNICIPIO' y 'ESTADO' es: {number_of_duplicated_rows}")
lista_beneficiarios[duplicated_rows]


El número de filas duplicadas en las columnas 'APELLIDO PATERNO', 'APELLIDO MATERNO', 'NOMBRE (S)', 'MUNICIPIO' y 'ESTADO' es: 38221


,ESTADO_prod,MUNICIPIO_prod,ACUSE ESTATAL,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE,KEY_benef_Verificado,Entidad_inegi,Municipio_inegi,CVE_ENT,CVE_MUN
2637,SINALOA,ANGOSTURA,23-PROESFE-ESTR-009922-E000-SL,ANGULO,PEREA,JAVIER,10.0,sinaloa-angostura,Sinaloa,Angostura,25,2
4888,SINALOA,BADIRAGUATO,23-PROESFE-ESTR-023990-E000-SL,LOPEZ,LOPEZ,FRANCISCO,2.0,sinaloa-badiraguato,Sinaloa,Badiraguato,25,3
5760,SINALOA,BADIRAGUATO,23-PROESFE-ESTR-023684-E000-SL,PEREZ,PEREZ,EMIGDIO,10.0,sinaloa-badiraguato,Sinaloa,Badiraguato,25,3
8203,SINALOA,CHOIX,23-PROESFE-ESTR-036610-E000-SL,PADILLA,REYES,ROSARIO,5.0,sinaloa-choix,Sinaloa,Choix,25,7
9349,SINALOA,CONCORDIA,23-PROESFE-ESTR-005179-E000-SL,CONTRERAS,LABRADOR,FRANCISCO,3.0,sinaloa-concordia,Sinaloa,Concordia,25,4
...,...,...,...,...,...,...,...,...,...,...,...,...
1680254,ZACATECAS,VILLA GARCIA,23-PRONAFE-FERT-049785-S000-ZS,LUGO,MUÑOZ,FRANCISCA,2.0,zacatecas-villa garcia,Zacatecas,Villa García,32,52
1680921,ZACATECAS,VILLA HIDALGO,23-PRONAFE-FERT-023799-S000-ZS,GUERRERO,DELGADO,VERONICA,2.0,zacatecas-villa hidalgo,Zacatecas,Villa Hidalgo,32,54
1681061,ZACATECAS,VILLA HIDALGO,23-PRONAFE-FERT-023982-S000-ZS,MARTINEZ,MARTINEZ,JOSE LUIS,2.0,zacatecas-villa hidalgo,Zacatecas,Villa Hidalgo,32,54
1681065,ZACATECAS,VILLA HIDALGO,23-PRONAFE-FERT-023988-S000-ZS,MARTINEZ,MARTINEZ,MA. DE LA LUZ,2.0,zacatecas-villa hidalgo,Zacatecas,Villa Hidalgo,32,54


## 3.4 Decisiones sobre valores duplicados

Dado que este número es muy grande, entendemos que se debe a que posiblemente los listados publicados no son únicos. Posiblemente contenían información repetida. Procederemos a eliminar las filas duplicadas.

In [ ]:
# Chunk vacío a propósito.



## 3.5 Corrección de Tipos de Datos

In [ ]:
lista_productores = lista_productores.astype({
    'ESTADO': 'str',
    'MUNICIPIO': 'str',
    'ACUSE': 'str',
    'APELLIDO PATERNO': 'str',
    'APELLIDO MATERNO': 'str',
    'NOMBRE (S)': 'str',
    'PAQUETE': 'int',
    'NOM_MUN': 'str',
    'NOM_ENT': 'str',
    'CVE_MUN': 'str',
    'CVE_ENT': 'str'
})

In [ ]:
lista_productores.info()

In [ ]:

lista_productores[['ESTADO_c', 'MUN_c']] = lista_productores['KEY_inegi'].str.split('-', expand=True)


# 4. Análisis Univariado

## 4.1 Variables Numéricas

En este caso, la única variable númerica es la de paquetes.

In [ ]:
# Filtrar los datos para estar dentro del rango de interés
paquetes = lista_productores['PAQUETE']

# Crear el histograma con un mayor número de bins
plt.figure(figsize=(10, 6))
barras = plt.hist(paquetes,bins=8,color='skyblue', edgecolor='black')
plt.xlabel('Numero de paquetes')
plt.ylabel('Cantidad')
plt.title('Histograma de Paquetes')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Recorrer las barras
for i in range(len(barras[0])):
    # Obtener el valor de la barra
    valor = barras[0][i]

     # Calcular la posición x de la anotación como el centro de la barra
    bar_width = barras[1][1] - barras[1][0]  # Ancho de las barras
    x_pos = barras[1][i] + bar_width / 2  # Posición x del centro de la barra

    # Mostrar el valor encima de la barra
    plt.annotate(f"{valor:.0f}", xy=(x_pos, valor),
                 ha="center", va="bottom", fontsize=8, color="black")

plt.show()

In [ ]:
# Contar el número de cada tipo de 'PAQUETE'
grouped_df = lista_productores['PAQUETE'].value_counts()

# Calcular el porcentaje relativo de cada tipo de paquete
percentage_df = (grouped_df / grouped_df.sum()) * 100

# Crear el gráfico de barras apiladas
percentage_df.plot(kind='barh', stacked=True, color='skyblue', edgecolor='black', figsize=(10, 6))

plt.xlabel('Porcentaje (%)')
plt.ylabel('Tipo de paquete')
plt.title('Porcentaje relativo a solicitud de cantidad de paquete')
plt.grid(axis='x', linestyle='--', alpha=0.7)

plt.show()

In [ ]:
# Contar el número de cada tipo de 'PAQUETE'
grouped_df = lista_productores['PAQUETE'].value_counts()

# Calcular el porcentaje relativo de cada tipo de paquete
percentage_df = (grouped_df / grouped_df.sum()) * 100

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
barras = plt.bar(percentage_df.index, percentage_df.values, color='skyblue', edgecolor='black')
plt.xlabel('Tipo de paquete')
plt.ylabel('Porcentaje (%)')
plt.title('Porcentaje relativo de cada tipo de paquete')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Añadir anotaciones a las barras
for bar in barras:
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
             f'{bar.get_height():.2f}%',
             ha='center', va='bottom')

plt.show()

## 4.2 Variables Categóricas

In [ ]:
# Agrupar los datos por 'ESTADO' y sumar el número de 'PAQUETE'
grouped_df = lista_productores.groupby("ESTADO_c")["PAQUETE"].sum()

# Ordenar los datos de forma descendente
grouped_df = grouped_df.sort_values(ascending=True)

# Crear el gráfico de barras horizontal
plt.figure(figsize=(10, 6))
barras = plt.barh(grouped_df.index, grouped_df.values, color='skyblue', edgecolor='black')
plt.xlabel('Cantidad de paquetes')
plt.ylabel('Estados')
plt.title('Cantidad de paquetes por Estado')
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Añadir anotaciones a las barras
for bar in barras:
    plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2,
             f'{bar.get_width():.0f}', 
             va='center', ha='left')

plt.show()

In [ ]:
# Agrupar los datos por 'ESTADO' y sumar el número de 'PAQUETE'
grouped_df = lista_productores.groupby("ESTADO_c")["PAQUETE"].count()

# Ordenar los datos de forma descendente
grouped_df = grouped_df.sort_values(ascending=True)

# Crear el gráfico de barras horizontal
plt.figure(figsize=(10, 6))
barras = plt.barh(grouped_df.index, grouped_df.values, color='skyblue', edgecolor='black')
plt.xlabel('Cantidad de paquetes')
plt.ylabel('Estados')
plt.title('Cantidad productores autorizados por Estado')
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Añadir anotaciones a las barras
for bar in barras:
    plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2,
             f'{bar.get_width():.0f}', 
             va='center', ha='left')

plt.show()

In [ ]:
# Estado y numero de paquetes filtrar los datos donde la solicitud de'PAQUETE' es 1
lista_productores_filtrada = lista_productores[lista_productores['PAQUETE'] == 1]

# Agrupar los datos filtrados por 'ESTADO' y contar el número de 'PAQUETE'
grouped_df = lista_productores_filtrada.groupby("ESTADO_c")["PAQUETE"].count()

# Ordenar los datos de forma descendente
grouped_df = grouped_df.sort_values(ascending=True)

# Crear el gráfico de barras horizontal
plt.figure(figsize=(10, 6))
barras = plt.barh(grouped_df.index, grouped_df.values, color='skyblue', edgecolor='black')
plt.xlabel('Cantidad de paquetes')
plt.ylabel('Estado')
plt.title('Cantidad productores autorizados por Estado')
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Añadir anotaciones a las barras
for bar in barras:
    plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2,
             f'{bar.get_width():.0f}', 
             va='center', ha='left')

plt.show()

In [ ]:
# Filtrar los datos donde 'PAQUETE' es 2
lista_productores_filtrada = lista_productores[lista_productores['PAQUETE'] == 2]

# Agrupar los datos filtrados por 'ESTADO' y contar el número de 'PAQUETE'
grouped_df = lista_productores_filtrada.groupby("ESTADO_c")["PAQUETE"].count()

# Ordenar los datos de forma descendente
grouped_df = grouped_df.sort_values(ascending=True)

# Crear el gráfico de barras horizontal
plt.figure(figsize=(10, 6))
barras = plt.barh(grouped_df.index, grouped_df.values, color='skyblue', edgecolor='black')
plt.xlabel('Cantidad de paquetes')
plt.ylabel('Estado')
plt.title('Cantidad de paquetes solicitados igual 2 por Estado')
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Añadir anotaciones a las barras
for bar in barras:
    plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2,
             f'{bar.get_width():.0f}', 
             va='center', ha='left')

plt.show()

In [ ]:
# Filtrar los datos donde 'PAQUETE' es 1 y 2
lista_productores_filtrada1 = lista_productores[lista_productores['PAQUETE'] == 1]
lista_productores_filtrada2 = lista_productores[lista_productores['PAQUETE'] == 2]

# Agrupar los datos filtrados por 'ESTADO' y contar el número de 'PAQUETE'
grouped_df1 = lista_productores_filtrada1.groupby("ESTADO_c")["PAQUETE"].count()
grouped_df2 = lista_productores_filtrada2.groupby("ESTADO_c")["PAQUETE"].count()

# Crear el gráfico de barras horizontal
plt.figure(figsize=(16, 6))

# Asegurarse de que los índices de los dos DataFrames son los mismos
index = np.union1d(grouped_df1.index, grouped_df2.index)
grouped_df1 = grouped_df1.reindex(index, fill_value=0)
grouped_df2 = grouped_df2.reindex(index, fill_value=0)

# Crear las barras para 'PAQUETE' igual a 1 y 2
barras1 = plt.barh(grouped_df1.index, grouped_df1.values, color='skyblue', edgecolor='black')
barras2 = plt.barh(grouped_df2.index, grouped_df2.values, color='orange', edgecolor='black', left=grouped_df1.values)

plt.xlabel('Cantidad de paquetes')
plt.ylabel('Estado')
plt.title('Cantidad de paquetes por Estado (separado por valor de PAQUETE)')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.legend(['PAQUETE = 1', 'PAQUETE = 2'])

# Añadir anotaciones a las barras
for bar1, bar2 in zip(barras1, barras2):
    plt.text(bar1.get_width(), bar1.get_y() + bar1.get_height()/2,
             f'{bar1.get_width():.0f}',
             va='center', ha='left')
    plt.text(bar1.get_width() + bar2.get_width(), bar2.get_y() + bar2.get_height()/2,
             f'{bar2.get_width():.0f}',
             va='center', ha='left')

In [ ]:
# Agrupar los datos por 'NOM_ENT' y 'PAQUETE', y contar el número de 'PAQUETE'
grouped_df = lista_productores.groupby(["ESTADO_c", "PAQUETE"]).size().unstack(fill_value=0)

# Calcular el porcentaje relativo de cada tipo de paquete por estado
percentage_df = grouped_df.div(grouped_df.sum(axis=1), axis=0) * 100

# Crear el gráfico de barras apiladas
fig, ax = plt.subplots(figsize=(10, 6))
percentage_df.plot(kind='barh', stacked=True, edgecolor='black', ax=ax)

plt.xlabel('Porcentaje de paquetes (%)')
plt.ylabel('Estado')
plt.title('Porcentaje relativo a la cantidad de paquetes solicitados por Estado')
plt.legend(title='PAQUETE', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Añadir anotaciones a las barras
for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    if width > 0:
        ax.text(x+width/2, 
                y+height/2, 
                '{:.0f} %'.format(width), 
                horizontalalignment='center', 
                verticalalignment='center')

plt.show()

# 5. Análisis Multivariado

In [ ]:
# Paquetes por municipio top20 
grouped_df = lista_productores.groupby("KEY_inegi")["PAQUETE"].count() 

# Ordenar los resultados en orden descendente y seleccionar los primeros 20
top20_df = grouped_df.sort_values(ascending=False).head(30)

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
barras = plt.barh(top20_df.index, top20_df.values, color='skyblue', edgecolor='black')
plt.xlabel('Suma de paquetes')
plt.ylabel('key_inegi')
plt.title('Top 30 key_inegi con la suma más alta de paquetes')
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Añadir anotaciones a las barras
for bar in barras:
    plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2,
             f'{bar.get_width():.0f}',
             va='center', ha='left')

plt.show()

In [ ]:
# Paquetes por municipio top20 
grouped_df = lista_productores.groupby("KEY_inegi")["PAQUETE"].count()

# Ordenar los resultados en orden descendente y seleccionar los primeros 20
top20_df = grouped_df.sort_values(ascending=False).head(30)

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
barras = plt.barh(top20_df.index, top20_df.values, color='skyblue', edgecolor='black')
plt.xlabel('Suma de paquetes')
plt.ylabel('key_inegi')
plt.title('Top 30 municipios con más productores autorizados')
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Añadir anotaciones a las barras
for bar in barras:
    plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2,
             f'{bar.get_width():.0f}',
             va='center', ha='left')

plt.show()

In [ ]:
# Saber que personas tienen mas paquetes
# Agrupar los datos por 'APELLIDO PATERNO', 'APELLIDO MATERNO', 'NOMBRE (S)' y sumar el número de 'PAQUETE'
filtered_df = lista_productores.loc[(lista_productores["APELLIDO PATERNO"] != 'unknown') & 
                                    (lista_productores["APELLIDO MATERNO"] != 'unknown') & 
                                    (lista_productores["NOMBRE (S)"] != 'unknown')]

# Agrupar los datos filtrados por 'APELLIDO PATERNO', 'APELLIDO MATERNO', 'NOMBRE (S)' y sumar el número de 'PAQUETE'
grouped_df = filtered_df.groupby(["APELLIDO PATERNO", "APELLIDO MATERNO", "NOMBRE (S)"])["PAQUETE"].sum() # Mostrar sin key_inegi

# Ordenar los resultados en orden descendente y seleccionar los primeros 20
top20_df = grouped_df.sort_values(ascending=False).head(20)

# Convertir el índice multi-nivel a una cadena
top20_df.index = top20_df.index.map(lambda x: ', '.join(x))

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
barras = plt.barh(top20_df.index, top20_df.values, color='skyblue', edgecolor='black')
plt.xlabel('Suma de paquetes')
plt.ylabel('Nombre')
plt.title('Top 20 nombres con la suma más alta de paquetes (excluyendo \'unknown\')')
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Añadir anotaciones a las barras
for bar in barras:
    plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2,
             f'{bar.get_width():.0f}',
             va='center', ha='left')

plt.show()

In [ ]:
 # Coincidencias de APELLIDO PATERNO, APELLIDO MATERNO y MUNICIPIO (Familias)
# Saber que personas tienen mas paquetes
# Agrupar los datos por 'APELLIDO PATERNO', 'APELLIDO MATERNO', 'NOMBRE (S)' y sumar el número de 'PAQUETE'
filtered_df = lista_productores.loc[(lista_productores["APELLIDO PATERNO"] != 'unknown') & 
                                    (lista_productores["APELLIDO MATERNO"] != 'unknown')]

# Agrupar los datos filtrados por 'APELLIDO PATERNO', 'APELLIDO MATERNO', 'NOMBRE (S)' y sumar el número de 'PAQUETE'
grouped_df = filtered_df.groupby(["APELLIDO PATERNO", "APELLIDO MATERNO",'KEY_inegi'])["PAQUETE"].sum() # Mostrar sin key_inegi

# Ordenar los resultados en orden descendente y seleccionar los primeros 20
top20_df = grouped_df.sort_values(ascending=False).head(20)

# Convertir el índice multi-nivel a una cadena
top20_df.index = top20_df.index.map(lambda x: ', '.join(x))

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
barras = plt.barh(top20_df.index, top20_df.values, color='skyblue', edgecolor='black')
plt.xlabel('Suma de paquetes')
plt.ylabel('Nombre')
plt.title('Top 20 nombres con la suma más alta de paquetes (excluyendo \'unknown\')')
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Añadir anotaciones a las barras
for bar in barras:
    plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2,
             f'{bar.get_width():.0f}',
             va='center', ha='left')

plt.show()